# Final version

In [68]:
import numpy as np
import random
import pandas as pd
import heapq
import time  # For measuring computation time


# Environment related code

In [69]:
class DisasterZoneEnv:
    """
    A simplified 2D grid environment for a drone exploring a disaster zone.

    Legend:
      0 -> Empty cell
      1 -> Obstacle
      2 -> Survivor
      3 -> Resource
      D -> Drone (tracked separately but displayed in render).
    """

    def __init__(self, width=8, height=8, num_obstacles=5, num_survivors=3, num_resources=2, initial_energy=20, dynamic=False):
        """
        Initialize the environment with configurable dimensions and grid contents.

        :param width: Width of the grid.
        :param height: Height of the grid.
        :param num_obstacles: Number of obstacles in the grid.
        :param num_survivors: Number of survivors in the grid.
        :param num_resources: Number of resources in the grid.
        :param initial_energy: Initial energy level of the drone.
        :param dynamic: Whether the environment changes dynamically over time.
        """
        self.width = width
        self.height = height
        self.num_obstacles = num_obstacles
        self.num_survivors = num_survivors
        self.num_resources = num_resources
        self.initial_energy = initial_energy
        self.dynamic = dynamic  # Enable or disable dynamic changes
        self.reset()

    def reset(self):
        """
        Resets the environment to its initial state.
        """
        # Create a blank grid
        self.grid = np.zeros((self.height, self.width), dtype=int)

        # Place obstacles, survivors, and resources randomly
        for _ in range(self.num_obstacles):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 1  # Obstacle

        for _ in range(self.num_survivors):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 2  # Survivor

        for _ in range(self.num_resources):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 3  # Resource

        # Place the drone at a random position
        self.drone_x, self.drone_y = self._get_random_empty_cell()

        # Set the drone's initial energy level
        self.energy = self.initial_energy

    def _get_random_empty_cell(self):
        """
        Finds a random empty cell (not occupied by an obstacle, survivor, or resource).
        """
        while True:
            x = random.randint(0, self.height - 1)
            y = random.randint(0, self.width - 1)
            if self.grid[x, y] == 0:  # Ensure the cell is empty
                return x, y

    def step(self, position):
        """
        Moves the drone to the specified position.

        :param position: Tuple (new_x, new_y) representing the target position.
        """
        new_x, new_y = position
        reward = 0
        done = False

        # Check if the new position is within bounds
        if not self._in_bounds(new_x, new_y):
            reward -= 10  # Penalize out-of-bounds moves
        elif self.grid[new_x, new_y] == 1:  # Obstacle collision
            reward -= 10
        else:
            # Valid move
            self.drone_x, self.drone_y = new_x, new_y
            if self.grid[new_x, new_y] == 2:  # Rescued a survivor
                reward += 10
                self.grid[new_x, new_y] = 0
            elif self.grid[new_x, new_y] == 3:  # Collected a resource
                reward += 5
                self.energy += 5  # Recharge energy
                self.grid[new_x, new_y] = 0
            else:  # Empty cell
                reward += 1

        # Decrease energy after the move
        self.energy -= 1
        reward -= 1  # Penalize energy usage
        if self.energy <= 0:
            done = True

        return reward, done

    def _in_bounds(self, x, y):
        """
        Checks if the position (x, y) is within the grid boundaries.
        """
        return 0 <= x < self.height and 0 <= y < self.width

    def render(self):
        """
        Visualizes the grid state.
        """
        grid_copy = self.grid.astype(str)
        grid_copy[grid_copy == '0'] = '.'  # Empty cell
        grid_copy[grid_copy == '1'] = '#'  # Obstacle
        grid_copy[grid_copy == '2'] = 'S'  # Survivor
        grid_copy[grid_copy == '3'] = 'R'  # Resource
        grid_copy[self.drone_x, self.drone_y] = 'D'  # Drone

        # Print the grid
        for row in grid_copy:
            print(" ".join(row))
        print(f"Energy: {self.energy}\n")


# Dijiksra Agent

In [70]:
class DijkstraAgent:
    """
    An agent that uses Dijkstra's algorithm to navigate the DisasterZoneEnv.
    """

    def __init__(self, env):
        self.env = env
        self.steps_taken = 0  # Counter for steps taken

    def dijkstra(self, grid, start, target):
        """
        Implements Dijkstra's algorithm to find the shortest path in a 2D grid.

        :param grid: The grid representing the environment.
        :param start: The starting position of the agent (x, y).
        :param target: The target position (x, y).
        :return: The shortest path as a list of coordinates and the total distance.
        """
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        pq = [(0, start)]  # Priority queue (distance, position)
        distances = {start: 0}
        previous = {start: None}

        while pq:
            current_distance, current_position = heapq.heappop(pq)
            if current_position == target:
                # Reconstruct path
                path = []
                while current_position:
                    path.append(current_position)
                    current_position = previous[current_position]
                path.reverse()
                return path, current_distance

            for dx, dy in directions:
                neighbor = (current_position[0] + dx, current_position[1] + dy)
                if 0 <= neighbor[0] < grid.shape[0] and 0 <= neighbor[1] < grid.shape[1]:
                    if grid[neighbor[0], neighbor[1]] != 1:  # Ignore obstacles
                        new_distance = current_distance + 1
                        if neighbor not in distances or new_distance < distances[neighbor]:
                            distances[neighbor] = new_distance
                            previous[neighbor] = current_position
                            heapq.heappush(pq, (new_distance, neighbor))
        return [], float('inf')  # No valid path

    def find_closest_target(self, target_type):
        """
        Finds the closest target (e.g., survivor or resource).

        :param target_type: 2 for survivors, 3 for resources.
        :return: Shortest path, target position, and distance to the closest target.
        """
        start = (self.env.drone_x, self.env.drone_y)
        targets = [(x, y) for x in range(self.env.height) for y in range(self.env.width) if self.env.grid[x, y] == target_type]

        if not targets:
            return None, None, float('inf')

        shortest_path, closest_target, shortest_distance = None, None, float('inf')
        for target in targets:
            path, distance = self.dijkstra(self.env.grid, start, target)
            if distance < shortest_distance:
                shortest_path, closest_target, shortest_distance = path, target, distance

        return shortest_path, closest_target, shortest_distance

    def execute(self):
        """
        Main execution loop for the agent.
        """
        while self.env.energy > 0:
            # Try to find a survivor first, then a resource
            path, target_position, distance = self.find_closest_target(2)
            if not path:  # No survivors, try finding a resource
                path, target_position, distance = self.find_closest_target(3)

            if not path:  # No reachable targets
                print("No reachable targets remaining. Stopping simulation.")
                break

            for step in path[1:]:  # Skip the first position (current position)
                self.env.step(step)  # Pass the tuple directly here (instead of unpacking with *step)
                self.steps_taken += 1
                if self.env.energy <= 0:
                    print("Drone ran out of energy!")
                    break

            # Clear the target from the grid
            if target_position:
                self.env.grid[target_position[0], target_position[1]] = 0

        print("Agent has completed its task.")


# Tester

In [71]:
class Tester:
    """
    A testing framework for evaluating agents in DisasterZoneEnv.
    """

    def __init__(self, env, num_tests=5):
        """
        Initialize the tester.

        :param env: An instance of DisasterZoneEnv.
        :param num_tests: Number of test runs to perform.
        """
        self.env = env
        self.num_tests = num_tests
        self.results = []  # Store results for all tests

    def run(self, agent_class):
        """
        Run tests for the specified agent class.

        :param agent_class: The agent class to test (e.g., DijkstraAgent).
        """
        for test_idx in range(self.num_tests):
            print(f"\n=== Running Test {test_idx + 1}/{self.num_tests} ===")

            # Reset the environment for each test
            self.env.reset()
            agent = agent_class(self.env)

            # Record initial metrics
            survivors_before = np.sum(self.env.grid == 2)
            resources_before = np.sum(self.env.grid == 3)
            energy_before = self.env.energy

            # Measure computation time
            start_time = time.time()
            agent.execute()
            end_time = time.time()

            # Record final metrics
            survivors_after = np.sum(self.env.grid == 2)
            resources_after = np.sum(self.env.grid == 3)
            energy_after = self.env.energy

            # Append results for this test
            self.results.append({
                "Test": test_idx + 1,
                "Grid Dimensions": f"{self.env.width}x{self.env.height}",
                "Obstacles": self.env.num_obstacles,
                "Survivors Rescued": survivors_before - survivors_after,
                "Resources Collected": resources_before - resources_after,
                "Energy Used": energy_before - energy_after,
                "Steps Taken": agent.steps_taken,
                "Computation Time (s)": round(end_time - start_time, 4),
            })

    def display_results(self):
        """
        Display the test results in a table format.
        """
        if not self.results:
            print("No results to display.")
            return

        # Convert results to a pandas DataFrame for tabular display
        df = pd.DataFrame(self.results)
        print("\nResults Table:")
        print(df)

    def save_results_to_csv(self, filename="test_results.csv"):
        """
        Save the test results to a CSV file.

        :param filename: Name of the CSV file to save the results.
        """
        if not self.results:
            print("No results to save.")
            return

        # Save results as a CSV file using pandas
        df = pd.DataFrame(self.results)
        df.to_csv(filename, index=False)
        print(f"Results saved to {filename}")


# Code

In [72]:
if __name__ == "__main__":
    # Create the environment
    env = DisasterZoneEnv(width=8, height=8, num_obstacles=5, num_survivors=3, num_resources=2, initial_energy=20, dynamic=False)

    # Initialize the tester
    tester = Tester(env, num_tests=3)  # Run 3 tests

    # Run the tests with the DijkstraAgent
    tester.run(agent_class=DijkstraAgent)

    # Display the results in the console
    tester.display_results()

    # Save results to a CSV file
    tester.save_results_to_csv("dijkstra_test_results.csv")



=== Running Test 1/3 ===
No reachable targets remaining. Stopping simulation.
Agent has completed its task.

=== Running Test 2/3 ===
No reachable targets remaining. Stopping simulation.
Agent has completed its task.

=== Running Test 3/3 ===
No reachable targets remaining. Stopping simulation.
Agent has completed its task.

Results Table:
   Test Grid Dimensions  Obstacles  Survivors Rescued  Resources Collected  \
0     1             8x8          5                  3                    2   
1     2             8x8          5                  3                    2   
2     3             8x8          5                  3                    2   

   Energy Used  Steps Taken  Computation Time (s)  
0            8           18                0.0009  
1           15           25                0.0009  
2           14           24                0.0011  
Results saved to dijkstra_test_results.csv
